In [1]:
import polars as pl
import pandas as pd
import geopandas as gpd

In [2]:
# odsの中身の確認
ods = pl.read_parquet('/Users/nagampere/File/Python/sayori/od_yonezawa_based_stop_id.parquet/part-00002-tid-7597389897474560305-7a8f602a-b508-4487-af54-55d4c76e5e85-38-1.c000.snappy.parquet')
ods.head(10)

d_key,o_key,datetime,hour,move_count,weekday_holiday_flag,o_mesh_4,o_stop_name,o_agency,d_mesh_4,d_stop_name,d_agency
str,str,datetime[ns],i32,i32,str,i64,str,str,i64,str,str
"""54_01""","""70_01""",2022-05-02 00:00:00,12,1,"""w""",564070091,"""金池一丁目""","""yonezawa_city""",564060873,"""西部こども園前""","""yonezawa_city"""
"""41_02""","""48_02""",2022-05-13 00:00:00,13,1,"""h""",564060982,"""上杉神社前""","""yonezawa_city""",564060893,"""県営相生ｱﾊﾟｰﾄ東""","""yonezawa_city"""
"""56_02""","""53_02""",2022-05-04 00:00:00,10,1,"""w""",564060874,"""愛宕小前""","""yonezawa_city""",564060973,"""西部ｺﾐｭﾆﾃｨｾﾝﾀｰ前""","""yonezawa_city"""
"""64_01""","""46_02""",2022-05-13 00:00:00,15,1,"""h""",564060884,"""大町一丁目""","""yonezawa_city""",564070083,"""信夫町""","""yonezawa_city"""
"""53_02""","""1_01""",2022-05-30 00:00:00,7,1,"""w""",564061384,"""米沢スキー場前""","""yonezawa_city""",564060874,"""愛宕小前""","""yonezawa_city"""
"""34_01""","""80_02""",2022-05-07 00:00:00,22,1,"""h""",564060693,"""太田町五丁目""","""yonezawa_city""",564070082,"""中央三丁目""","""yonezawa_city"""
"""52_01""","""58_01""",2022-05-26 00:00:00,21,1,"""h""",564070072,"""すこやかセンター""","""yonezawa_city""",564060872,"""二中前""","""yonezawa_city"""
"""12_02""","""27_02""",2022-05-31 00:00:00,17,1,"""w""",564060992,"""米沢営業所""","""yonezawa_city""",564061831,"""万世小学校前""","""yonezawa_city"""
"""86_02""","""52_02""",2022-05-16 00:00:00,16,1,"""w""",564060872,"""二中前""","""yonezawa_city""",564060884,"""九里学園前""","""yonezawa_city"""


In [3]:
import json
ods_parameters = ods.with_columns(
    pl.struct(
        pl.concat_list(pl.col('o_key')).alias("origin_stop_ids"),
        # pl.concat_list(pl.lit("26_01")).alias("origin_stop_ids"),
        pl.concat_list(pl.col('d_key')).alias("destination_stop_ids"),
        # pl.concat_list(pl.lit("42_01")).alias("destination_stop_ids"),
        pl.col('datetime').cast(pl.Date).cast(str).alias("specified_date"),
        # pl.lit("2022-06-20").alias("specified_date"),
        (pl.col('hour')*60*60).alias("specified_secs"),
        # (pl.lit('80000')).alias("specified_secs"),
        pl.lit(0).alias("transfers_limit"),
        pl.lit(True).alias("is_reverse_search"),
        pl.lit(None).cast(str).alias("available_trip_ids")
    ).alias("parameters")
)
print(ods_parameters[0,12])
ods_parameters.head(10)


{'origin_stop_ids': ['70_01'], 'destination_stop_ids': ['54_01'], 'specified_date': '2022-05-02', 'specified_secs': 43200, 'transfers_limit': 0, 'is_reverse_search': True, 'available_trip_ids': None}


d_key,o_key,datetime,hour,move_count,weekday_holiday_flag,o_mesh_4,o_stop_name,o_agency,d_mesh_4,d_stop_name,d_agency,parameters
str,str,datetime[ns],i32,i32,str,i64,str,str,i64,str,str,struct[7]
"""54_01""","""70_01""",2022-05-02 00:00:00,12,1,"""w""",564070091,"""金池一丁目""","""yonezawa_city""",564060873,"""西部こども園前""","""yonezawa_city""","{[""70_01""],[""54_01""],""2022-05-02"",43200,0,true,null}"
"""41_02""","""48_02""",2022-05-13 00:00:00,13,1,"""h""",564060982,"""上杉神社前""","""yonezawa_city""",564060893,"""県営相生ｱﾊﾟｰﾄ東""","""yonezawa_city""","{[""48_02""],[""41_02""],""2022-05-13"",46800,0,true,null}"
"""56_02""","""53_02""",2022-05-04 00:00:00,10,1,"""w""",564060874,"""愛宕小前""","""yonezawa_city""",564060973,"""西部ｺﾐｭﾆﾃｨｾﾝﾀｰ前""","""yonezawa_city""","{[""53_02""],[""56_02""],""2022-05-04"",36000,0,true,null}"
"""64_01""","""46_02""",2022-05-13 00:00:00,15,1,"""h""",564060884,"""大町一丁目""","""yonezawa_city""",564070083,"""信夫町""","""yonezawa_city""","{[""46_02""],[""64_01""],""2022-05-13"",54000,0,true,null}"
"""53_02""","""1_01""",2022-05-30 00:00:00,7,1,"""w""",564061384,"""米沢スキー場前""","""yonezawa_city""",564060874,"""愛宕小前""","""yonezawa_city""","{[""1_01""],[""53_02""],""2022-05-30"",25200,0,true,null}"
"""34_01""","""80_02""",2022-05-07 00:00:00,22,1,"""h""",564060693,"""太田町五丁目""","""yonezawa_city""",564070082,"""中央三丁目""","""yonezawa_city""","{[""80_02""],[""34_01""],""2022-05-07"",79200,0,true,null}"
"""52_01""","""58_01""",2022-05-26 00:00:00,21,1,"""h""",564070072,"""すこやかセンター""","""yonezawa_city""",564060872,"""二中前""","""yonezawa_city""","{[""58_01""],[""52_01""],""2022-05-26"",75600,0,true,null}"
"""12_02""","""27_02""",2022-05-31 00:00:00,17,1,"""w""",564060992,"""米沢営業所""","""yonezawa_city""",564061831,"""万世小学校前""","""yonezawa_city""","{[""27_02""],[""12_02""],""2022-05-31"",61200,0,true,null}"
"""86_02""","""52_02""",2022-05-16 00:00:00,16,1,"""w""",564060872,"""二中前""","""yonezawa_city""",564060884,"""九里学園前""","""yonezawa_city""","{[""52_02""],[""86_02""],""2022-05-16"",57600,0,true,null}"


In [4]:
# transferデータの修正
stops = pd.read_csv('data_yonezawa/stops.txt')

In [15]:
pdf_stops = gpd.GeoDataFrame(
    stops,
    geometry=gpd.GeoSeries.from_xy(stops['stop_lon'], 
                                   stops['stop_lat'],
                                   crs="EPSG:4326")
).to_crs(6678)

s = gpd.GeoSeries.from_xy(stops['stop_lon'],
                          stops['stop_lat'],
                          crs="EPSG:4326").to_crs(6678)

pdf_stops = pdf_stops.sjoin_nearest(pdf_stops[['stop_id','stop_name','geometry']],
                                    how='left',
                                    max_distance=80*30,
                                    distance_col='distance',
                                    exclusive = True)
pdf_stops = pdf_stops[pdf_stops['stop_name_left']!=pdf_stops['stop_name_right']]
pdf_stops = pdf_stops.dropna(subset=['stop_name_right'])

In [21]:
transfers = pd.DataFrame()
transfers['from_stop_id'] = pdf_stops['stop_id_left'].astype(str)
transfers['to_stop_id'] = pdf_stops['stop_id_right'].astype(str)
transfers['transfer_type'] = 2
transfers['transfer_type'] = transfers['transfer_type'].astype('i4')
transfers['min_transfer_time'] = pdf_stops['distance'] // 80
transfers['min_transfer_time'] = transfers['min_transfer_time'].apply(lambda x: 1 if x == 0 else x)
transfers['min_transfer_time'] = transfers['min_transfer_time'].astype('i4')
transfers.reset_index(inplace=True,
                      drop=True)
transfers.to_parquet('data_yonezawa/transfers.parquet')
transfers.dtypes

from_stop_id         object
to_stop_id           object
transfer_type         int32
min_transfer_time     int32
dtype: object

In [17]:
import os
import time
import json

from sayori.raptor import search_p2p_geojson, search_p2p_path
from sayori.models import FeedPath, Feed

In [18]:
def load_data_from_path() -> FeedPath:
    feed_path = {
        "stops": os.environ.get("STOPS_PATH"),
        "trips": os.environ.get("TRIPS_PATH"),
        "stop_times": os.environ.get("STOP_TIMES_PATH"),
        "transfers": os.environ.get("TRANSFERS_PATH"),
        "calendar": os.environ.get("CALENDAR_PATH"),
    }

    return FeedPath.parse_obj(feed_path)

feed_path = load_data_from_path()
feed = Feed.from_feed_path(feed_path)

In [19]:
# pd_ods = ods_parameters.to_pandas()
ods_parameters_res = ods_parameters.with_columns(
    pl.col("parameters").apply(lambda x:str(search_p2p_geojson(feed,x))).alias("res")
)
# pl.col("parameters").apply(lambda x: x['parameters'])
req = {
        "origin_stop_ids": ["26_01"],
        "destination_stop_ids": ["42_01"],
        "specified_date": "2022-06-20",
        "specified_secs": 80000,
        "transfers_limit": 0,
        "is_reverse_search": True,
        "available_trip_ids": None
    }
res = ods_parameters["parameters"][0]
print(type(ods_parameters["parameters"][0]["origin_stop_ids"]))
print(type(req["origin_stop_ids"]))
res = search_p2p_geojson(feed, req)
print(res)
# test = ods_parameters.with_columns(pl.lit(str(res)).alias('test'))
ods_parameters_res
# .filter(pl.col("res").is_not_null())

<class 'list'>
<class 'list'>
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'LineString', 'coordinates': [[140.128138, 37.909013], [140.12512, 37.909219], [140.122879, 37.909106], [140.117623, 37.907855], [140.118408, 37.905467]]}, 'properties': {'time_to_reach': 15500, 'routing_path': ['26_01', '38_01', '39_01', '40_01', '42_01'], 'routing_path_optional': array([('2全日冬期_06時45分_系統4181', 1, '26_01'),
       ('2全日冬期_06時45分_系統4181', 2, '38_01'),
       ('2全日冬期_06時45分_系統4181', 3, '39_01'),
       ('2全日冬期_06時45分_系統4181', 4, '40_01'),
       ('2全日冬期_06時45分_系統4181', 5, '42_01')],
      dtype=[('trip_id', 'O'), ('stop_sequence', '<i8'), ('stop_id', 'O')]), 'preceding': ['2全日冬期_06時45分_系統4181'], 'stop_id': '26_01'}}]}


d_key,o_key,datetime,hour,move_count,weekday_holiday_flag,o_mesh_4,o_stop_name,o_agency,d_mesh_4,d_stop_name,d_agency,parameters,res
str,str,datetime[ns],i32,i32,str,i64,str,str,i64,str,str,struct[7],str
"""54_01""","""70_01""",2022-05-02 00:00:00,12,1,"""w""",564070091,"""金池一丁目""","""yonezawa_city""",564060873,"""西部こども園前""","""yonezawa_city""","{[""70_01""],[""54_01""],""2022-05-02"",43200,0,true,null}","""None"""
"""41_02""","""48_02""",2022-05-13 00:00:00,13,1,"""h""",564060982,"""上杉神社前""","""yonezawa_city""",564060893,"""県営相生ｱﾊﾟｰﾄ東""","""yonezawa_city""","{[""48_02""],[""41_02""],""2022-05-13"",46800,0,true,null}","""None"""
"""56_02""","""53_02""",2022-05-04 00:00:00,10,1,"""w""",564060874,"""愛宕小前""","""yonezawa_city""",564060973,"""西部ｺﾐｭﾆﾃｨｾﾝﾀｰ前""","""yonezawa_city""","{[""53_02""],[""56_02""],""2022-05-04"",36000,0,true,null}","""None"""
"""64_01""","""46_02""",2022-05-13 00:00:00,15,1,"""h""",564060884,"""大町一丁目""","""yonezawa_city""",564070083,"""信夫町""","""yonezawa_city""","{[""46_02""],[""64_01""],""2022-05-13"",54000,0,true,null}","""None"""
"""53_02""","""1_01""",2022-05-30 00:00:00,7,1,"""w""",564061384,"""米沢スキー場前""","""yonezawa_city""",564060874,"""愛宕小前""","""yonezawa_city""","{[""1_01""],[""53_02""],""2022-05-30"",25200,0,true,null}","""None"""
"""34_01""","""80_02""",2022-05-07 00:00:00,22,1,"""h""",564060693,"""太田町五丁目""","""yonezawa_city""",564070082,"""中央三丁目""","""yonezawa_city""","{[""80_02""],[""34_01""],""2022-05-07"",79200,0,true,null}","""None"""
"""52_01""","""58_01""",2022-05-26 00:00:00,21,1,"""h""",564070072,"""すこやかセンター""","""yonezawa_city""",564060872,"""二中前""","""yonezawa_city""","{[""58_01""],[""52_01""],""2022-05-26"",75600,0,true,null}","""None"""
"""12_02""","""27_02""",2022-05-31 00:00:00,17,1,"""w""",564060992,"""米沢営業所""","""yonezawa_city""",564061831,"""万世小学校前""","""yonezawa_city""","{[""27_02""],[""12_02""],""2022-05-31"",61200,0,true,null}","""None"""
"""86_02""","""52_02""",2022-05-16 00:00:00,16,1,"""w""",564060872,"""二中前""","""yonezawa_city""",564060884,"""九里学園前""","""yonezawa_city""","{[""52_02""],[""86_02""],""2022-05-16"",57600,0,true,null}","""None"""


In [20]:
print(len(ods_parameters_res))
print(len(ods_parameters_res.filter(pl.col("res")!='None')))

20480
486


res
str
"""{'type': 'Feat…"
"""{'type': 'Feat…"
"""{'type': 'Feat…"
"""{'type': 'Feat…"
"""{'type': 'Feat…"
"""{'type': 'Feat…"
"""{'type': 'Feat…"
"""{'type': 'Feat…"
"""{'type': 'Feat…"
